In [540]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_regression, SelectKBest, VarianceThreshold
import statsmodels.api as sm

In [615]:
distance_dums = pd.get_dummies(df['dist_cats'], drop_first=True)

In [616]:
carrier_dums = pd.get_dummies(df['op_unique_carrier'], drop_first=True)

In [626]:
Rscaler = RobustScaler()
robust_scaled = pd.DataFrame(Rscaler.fit_transform(polyt))
Sscaler = StandardScaler()
standard_scaled = pd.DataFrame(Sscaler.fit_transform(polyt))

In [314]:
robust_scaled.columns = X.columns
standard_scaled.columns = X.columns

In [451]:
X_train, X_test, y_train, y_test = train_test_split(t, y, test_size=0.2, random_state=0)

In [458]:
f_regression(X_train, y_train)[1]

array([2.34892628e-081, 9.38861944e-060, 5.76631731e-093, 3.04702407e-065,
       8.76825224e-046, 5.97955621e-019, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 1.48348552e-002, 2.77405841e-028,
       0.00000000e+000, 4.88979293e-128, 1.31446939e-035, 8.38672876e-004,
       1.19270348e-019])

In [194]:
reg = LinearRegression()

In [452]:
reg.fit(X_train, y_train)    

LinearRegression()

In [453]:
reg.score(X_test, y_test)

0.4299953926383264

In [614]:
depdum = pd.get_dummies(df['dep_hour'], drop_first=True)
arrdum = pd.get_dummies(df['arr_hour'], drop_first=True)
weekdum = pd.get_dummies(df['weekday'], drop_first=True)

In [212]:
X = pd.concat([distance_dums, carrier_dums, robust_scaled[0], weekdum, arrdum, origin_airports, departure_airports], axis=1)

In [492]:
weekdum.columns = ['mon', 'tue', 'wed', 'thur', 'fri', 'sat', 'sun']

In [172]:
y = df['arr_delay']

In [403]:
hourly_arr_delay = df.groupby(['dest_airport_id', 'weekday', 'arr_hour'], as_index=False).agg({'arr_delay': 'mean'})

In [404]:
hourly_arr_delay = hourly_arr_delay.rename(columns={'arr_delay': 'hourly_arr_delay'})
hourly_dep_delay = hourly_dep_delay.rename(columns={'dep_delay': 'hourly_dep_delay'})

In [456]:
reg.coef_

array([-0.21148488,  0.02382765, -0.0597098 , -0.14823941, -0.11505665,
        0.26469984,  0.69218393,  0.81376967,  0.59369118,  0.04840041,
       -0.42302682, -0.54559768,  0.9105955 , -0.56719821, -0.0195271 ,
        0.10442781, -0.08490071])

In [630]:
df.to_csv('../EDA_and_preprocessing/final_process.csv', index=False)

In [471]:
X = X[['crs_elapsed_time', 'hourly_arr_delay', 'hourly_dep_delay', 'tail_arr_delay', 'tail_dep_delay', 'tail_taxi_total', 'airport_taxi_total']]

In [632]:
#test = sm.add_constant(X)
mod = sm.OLS(y, X)
print(mod.fit().summary())

                                 OLS Regression Results                                
Dep. Variable:              arr_delay   R-squared (uncentered):                   0.434
Model:                            OLS   Adj. R-squared (uncentered):              0.434
Method:                 Least Squares   F-statistic:                              7552.
Date:                Tue, 12 Apr 2022   Prob (F-statistic):                        0.00
Time:                        13:19:22   Log-Likelihood:                     -4.9521e+05
No. Observations:              108140   AIC:                                  9.904e+05
Df Residuals:                  108129   BIC:                                  9.905e+05
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [617]:
test = pd.concat([standard_scaled, distance_dums, carrier_dums],axis=1)

In [622]:
poly = PolynomialFeatures(degree=2)
polyt = poly.fit_transform(X)

In [623]:
ploycols = poly.get_feature_names_out(X.columns)
pd.DataFrame(polyt, columns=ploycols)

,1,tail_taxi_total,taxi_total,airport_taxi_out_avg,airport_taxi_in_avg,tail_dep_delay,tail_arr_delay,crs_elapsed_time,tail_speed,hourly_arr_delay,...,tail_speed^2,tail_speed hourly_arr_delay,tail_speed hourly_dep_delay,tail_speed speed,hourly_arr_delay^2,hourly_arr_delay hourly_dep_delay,hourly_arr_delay speed,hourly_dep_delay^2,hourly_dep_delay speed,speed^2
0,1.0,21.869565,16.0,18.816138,5.884615,-0.086957,-9.304348,81.0,6.140610,-4.100000,...,37.707095,-25.176502,102.855223,29.146277,16.810000,-68.675000,-19.460563,280.562500,79.503521,22.529062
1,1.0,21.869565,22.0,18.816138,5.884615,-0.086957,-9.304348,82.0,6.140610,-6.444444,...,37.707095,-39.572822,92.345332,26.875139,41.530864,-96.914530,-28.204906,226.155325,65.817682,19.154832
2,1.0,21.869565,23.0,18.816138,5.884615,-0.086957,-9.304348,81.0,6.140610,0.777778,...,37.707095,4.776030,-2.686517,25.867321,0.604938,-0.340278,3.276389,0.191406,-1.842969,17.745156
3,1.0,21.869565,16.0,18.816138,5.884615,-0.086957,-9.304348,88.0,6.140610,-8.000000,...,37.707095,-49.124883,45.884005,27.964671,64.000000,-59.777778,-36.432432,55.834105,34.028904,20.739408
4,1.0,21.869565,19.0,18.816138,8.425434,-0.086957,-9.304348,260.0,6.140610,-8.520833,...,37.707095,-52.323117,0.731025,51.093027,72.604601,-1.014385,-70.897703,0.014172,0.990537,69.230934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108135,1.0,12.812500,10.0,8.416667,4.375000,-10.625000,-8.750000,50.0,2.370632,-8.500000,...,5.619895,-20.150370,-17.779738,6.116230,72.250000,63.750000,-21.930000,56.250000,-19.350000,6.656400
108136,1.0,12.812500,10.0,8.416667,4.375000,-10.625000,-8.750000,50.0,2.370632,-25.000000,...,5.619895,-59.265794,-59.265794,6.116230,625.000000,625.000000,-64.500000,625.000000,-64.500000,6.656400
108137,1.0,12.812500,15.0,8.416667,4.375000,-10.625000,-8.750000,50.0,2.370632,-25.000000,...,5.619895,-59.265794,-71.118953,5.560209,625.000000,750.000000,-58.636364,900.000000,-70.363636,5.501157
108138,1.0,12.812500,10.0,8.416667,4.375000,-10.625000,-8.750000,50.0,2.370632,0.000000,...,5.619895,0.000000,11.853159,6.795811,0.000000,0.000000,0.000000,25.000000,14.333333,8.217778


In [554]:
df['speed'] = df['distance'] / df['actual_elapsed_time']
speed = df.groupby('tail_num', as_index=False).agg({'speed': 'mean'})
speed = speed.rename(columns={'speed': 'tail_speed'})
df = df.merge(speed, on='tail_num')

In [631]:
X = df[['tail_taxi_total', 'taxi_total', 'airport_taxi_out_avg', 'airport_taxi_in_avg', 'tail_dep_delay', 'tail_arr_delay', 'crs_elapsed_time', 'tail_speed', 'hourly_arr_delay', 'hourly_dep_delay', 'speed']]

In [580]:
depdummies = pd.get_dummies(df['dep_hour'], drop_first=True)
arrdummies = pd.get_dummies(df['arr_hour'], drop_first=True)

In [628]:
X = pd.concat([depdummies, arrdummies, robust_scaled, weekdum, carrier_dums], axis=1)

In [598]:
X['crs_elapsed_time'] = X['crs_elapsed_time'].replace({0: 1})

C:\Users\broug\AppData\Local\Temp\ipykernel_18408\1928365387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['crs_elapsed_time'] = X['crs_elapsed_time'].replace({0: 1})


In [599]:
X['crs_elapsed_time'] = X['crs_elapsed_time'].apply(np.log)

C:\Users\broug\AppData\Local\Temp\ipykernel_18408\1040022879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['crs_elapsed_time'] = X['crs_elapsed_time'].apply(np.log)
